In [ ]:
#创建jobs.sql文件，用于创建jobs表
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker

# 创建数据库引擎，'sqlite:///jobs.db' 指定了数据库的路径为当前目录下的 jobs.db 文件
engine = create_engine('sqlite:///jobs.db')

# 创建元数据实例
metadata = MetaData()

# 定义表
jobs = Table('jobs', metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('job_id', String(50), unique=True),
    Column('task', String(10)),
    Column('ip', String(50)),
    Column('num_sequence', Integer),
    Column('status', String(30)),
    Column('email', String(30)),
    Column('submit_time', DateTime),
)

# 创建表
metadata.create_all(engine)

In [ ]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# 创建数据库引擎，'sqlite:///jobs.db' 指定了数据库的路径为当前目录下的 jobs.db 文件
engine = create_engine('sqlite:///jobs.db')

# 创建元数据实例
metadata = MetaData()

# 获取 jobs 表对象
jobs = Table('jobs', metadata)

# 创建数据库会话
Session = sessionmaker(bind=engine)
session = Session()

# 执行删除操作
session.query(jobs).delete()

# 提交事务
session.commit()

# 关闭会话
session.close()

In [11]:
import os

# 遍历./dpos_db/valid_dpos目录下的所有子目录，将每个子目录中的sequence.fasta文件写入到published_dpos.fasta文件中
def write_published_dpos():
    with open('published_dpos.fasta', 'w') as f:
        for root, dirs, files in os.walk('./dpos_db/valid_dpos'):
            for file in files:
                if file == 'sequence.fasta':
                    with open(os.path.join(root, file), 'r') as sequence:
                        f.write(sequence.read())
                        f.write('\n')

write_published_dpos()




In [7]:
from Bio import Entrez, SeqIO

Entrez.email = 'shenyanxiang@sjtu.edu.cn'

# 读取包含蛋白质 accession 号的文件
with open('dpos_database.txt', 'r') as file:
    protein_accessions = file.read().splitlines()

# 根据蛋白质 accession 号获取对应的基因组 accession 号
genome_accessions = []
for protein_accession in protein_accessions:
    handle = Entrez.efetch(db='protein', id=protein_accession, rettype='gb', retmode='text')
    record = SeqIO.read(handle, 'genbank')
    genome_accession = record.annotations['db_source'].split(' ')[-1]
    genome_accessions.append(genome_accession)

for genome_accession in genome_accessions:
#写入文件
    with open('dpos_genome_accessions.txt', 'a') as f:
        f.write(genome_accession)
        f.write('\n')

In [2]:
import os

with open('dpos_database.txt', 'r') as file:
    protein_accessions = file.read().splitlines()

for protein_accession in protein_accessions:
    #检查./dpos_db/valid_dpos目录下是否存在以蛋白质accession号命名的子目录
    if not os.path.exists(f'./dpos_db/valid_dpos/{protein_accession}'):
        print(f'./dpos_db/valid_dpos/{protein_accession} does not exist')


In [5]:
#读取published_dpos_genome.tsv
import pandas as pd
from Bio import Entrez, SeqIO
import os

published_dpos_genome = pd.read_csv('published_dpos_genome.tsv', sep='\t', header=None)

published_dpos_genome.columns = ['protein_accession', 'genome_accession']

#接下来的任务是遍历这个表，根据每行的protein_accession,进入/public/yxshen/DposFinderWeb/server/dpos_db/valid_dpos/protein_accession目录下,然后从
#NCBI下载对应的基因组序列，保存为genome.gb文件

Entrez.email = 'shenyanxiang@sjtu.edu.cn'

for index, row in published_dpos_genome.iterrows():
    protein_accession = row['protein_accession']
    genome_accession = row['genome_accession']
    if not os.path.exists(f'./dpos_db/valid_dpos/{protein_accession}'):
        print(f'./dpos_db/valid_dpos/{protein_accession} does not exist')
    else:
        handle = Entrez.efetch(db='nucleotide', id=genome_accession, rettype='gb', retmode='text')
        record = SeqIO.read(handle, 'genbank')
        with open(f'./dpos_db/valid_dpos/{protein_accession}/genome.gb', 'w') as f:
            SeqIO.write(record, f, 'genbank')

In [9]:
import os
import pandas as pd
from Bio import SeqIO

published_dpos_genome = pd.read_csv('published_dpos_genome.tsv', sep='\t', header=None)
published_dpos_genome.columns = ['protein_accession', 'genome_accession']

for index, row in published_dpos_genome.iterrows():
    protein_accession = row['protein_accession']
    genome_accession = row['genome_accession']
    genome_features = pd.DataFrame(columns=['id', 'protein_id', 'annotation', 'locus_tag', 'gene', 'start', 'end', 'strand', 'accession'])
    if not os.path.exists(f'./dpos_db/valid_dpos/{protein_accession}'):
        print(f'./dpos_db/valid_dpos/{protein_accession} does not exist')
    else:
        with open(f'./dpos_db/valid_dpos/{protein_accession}/genome.gb', 'r') as f:
            record = SeqIO.read(f, 'genbank')
            id_counter = 1  # 计数器变量
            for feature in record.features:
                if feature.type == 'CDS':
                    protein_id = feature.qualifiers['protein_id'][0] if 'protein_id' in feature.qualifiers else ''
                    annotation = feature.qualifiers['product'][0] if 'product' in feature.qualifiers else ''
                    locus_tag = feature.qualifiers['locus_tag'][0] if 'locus_tag' in feature.qualifiers else f'gp{id_counter}'
                    gene = feature.qualifiers['gene'][0] if 'gene' in feature.qualifiers else ''
                    start = feature.location.start
                    end = feature.location.end
                    strand = feature.location.strand
                    accession = genome_accession
                    genome_features = genome_features.append({'id': id_counter, 'protein_id': protein_id, 'annotation': annotation, 'locus_tag': locus_tag, 'gene': gene, 'start': start, 'end': end, 'strand': strand, 'accession': accession}, ignore_index=True)
                    id_counter += 1

    genome_features.to_csv(f'./dpos_db/valid_dpos/{protein_accession}/genome_features.csv', index=False)



In [ ]:
import os
import pandas as pd

published_dpos_genome = pd.read_csv('published_dpos_genome.tsv', sep='\t', header=None)
published_dpos_genome.columns = ['protein_accession', 'genome_accession']

published_dpos_genome = published_dpos_genome[published_dpos_genome['protein_accession'] != 'NP_690860.1']

for index, row in published_dpos_genome.iterrows():
    protein_accession = row['protein_accession']
    print(protein_accession)
    genome_accession = row['genome_accession']
    features = pd.read_csv(f'./dpos_db/valid_dpos/{protein_accession}/genome_features.csv')
    center = features[features['protein_id'].apply(lambda x: x.split('.')[0]) == protein_accession.split('.')[0]]['id']
    center = int(center)
    
    context = features[abs(features['id'] - center) <= 5]

    names = [f'{locus_tag} ({gene})' if isinstance(gene, str) else locus_tag for gene, locus_tag in zip(
            context['gene'], context['locus_tag'])]
    orietations = [1 if strand == '+' else -1 for strand in context['strand']]
    colors = ['red' if id == center else 'gray' for id in context['id']]

    context.loc[:, 'name'] = names
    context.loc[:, 'orietation'] = orietations
    context.loc[:, 'color'] = colors

    context.to_csv(f'./dpos_db/valid_dpos/{protein_accession}/genes.csv', index=False)



In [7]:
import os
import pandas as pd
import subprocess

published_dpos_genome = pd.read_csv('published_dpos_genome.tsv', sep='\t', header=None)
published_dpos_genome.columns = ['protein_accession', 'genome_accession']

published_dpos_genome = published_dpos_genome[published_dpos_genome['protein_accession'] != 'NP_690860.1']

#将protein_accession号写入accession.tsv文件

published_dpos_genome['protein_accession'].to_csv('accession.tsv', sep='\t', index=False, header=False)
